# Production Technology

The dataset contains `N = 441` firms observed over `T = 12` years, 1968-1979. There variables are: 
* `lcap`: Log of capital stock, $k_{it}$ 
* `lemp`: log of employment, $\ell_{it}$ 
* `ldsa`: log of deflated sales, $y_{it}$
* `year`: the calendar year of the observation, `year` $ = 1968, ..., 1979$, 
* `firmid`: anonymized indicator variable for the firm, $i = 1, ..., N$, with $N=441$. 

In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns
from scipy.stats import chi2
import numpy.linalg as la
from numpy import linalg as la
import scipy.stats as st
import importlib.util, sys


In [2]:
# --- loader ---
def load_firm_data(path="firms.csv"):
    dat = pd.read_csv(path)
    y = dat["ldsa"].values.reshape(-1, 1)
    x = np.column_stack([np.ones(dat.shape[0]), dat["lcap"].values, dat["lemp"].values])
    T = dat["year"].nunique()  # 12
    label_y = "Log deflated sales"
    label_x = ["Constant", "Capital", "Labor"]
    return y, x, T, dat["year"].values, label_y, label_x



In [3]:
import w3_LinearModels as lm

# Load firm data
y, x, T, year, label_y, label_x = load_firm_data()

# Pooled OLS estimation
pols_result = lm.estimate(y, x, T=T)

# Print results in a nice table
lm.print_table((label_y, label_x), pols_result, title="Pooled OLS", floatfmt='.4f')


Pooled OLS
Dependent variable: Log deflated sales

            Beta      Se    t-values
--------  ------  ------  ----------
Constant  0.0000  0.0050      0.0000
Capital   0.3100  0.0091     33.9237
Labor     0.6748  0.0102     66.4625
R² = 0.914
σ² = 0.131


In [4]:
pols_result_robust = lm.estimate(y, x, T=T, robust_se=True)
lm.print_table((label_y, label_x), pols_result_robust, title="Pooled OLS (Robust SE)", floatfmt='.4f')

Pooled OLS (Robust SE)
Dependent variable: Log deflated sales

            Beta      Se    t-values
--------  ------  ------  ----------
Constant  0.0000  0.0161      0.0000
Capital   0.3100  0.0324      9.5810
Labor     0.6748  0.0366     18.4526
R² = 0.914
σ² = 0.131


In [5]:
def remove_zero_columns(x, label_x, tol=1e-10):
    """
    Remove columns that are effectively zero and return filtered matrix and labels.

    Args:
        x: numpy array with regressors.
        label_x: list of labels for each column in x.
        tol: numerical tolerance for treating a column as zero (default 1e-10).

    Returns:
        Tuple of (x_nonzero, label_nonzero).
    """
    mask = ~np.all(np.isclose(x, 0.0, atol=tol), axis=0)
    x_nonzero = x[:, mask]
    label_nonzero = [label_x[i] for i, keep in enumerate(mask) if keep]
    return x_nonzero, label_nonzero


In [6]:
# --- CRS restriction helpers ---
def _crs_wald(results, start_idx):
    """Generic Wald test for the CRS restriction beta_K + beta_L = 1."""
    b = results['b_hat'][start_idx:, :]
    cov = results['cov'][start_idx:, start_idx:]
    R = np.array([[1.0, 1.0]])
    q = np.array([[1.0]])
    diff = R @ b - q
    var_Rb = R @ cov @ R.T
    W = (diff.T @ la.inv(var_Rb) @ diff).item()
    crit_val = chi2.ppf(0.95, 1)
    p_value = 1 - st.chi2.cdf(W, 1)
    return W, crit_val, p_value

def crs_test(results):
    """CRS test for models that include a constant (skip the first coefficient)."""
    return _crs_wald(results, 1)

def crs_test_fd(results):
    """CRS test for first-difference models without a constant."""
    return _crs_wald(results, 0)


In [7]:
# Transform the data
Q_T = np.eye(T) - 1/T * np.ones((T, T))
y_dot = lm.perm(Q_T, y)
x_dot = lm.perm(Q_T, x)

# Remove the columns that are only zeroes
x_dot, label_x_dot = remove_zero_columns(x_dot, label_x)

# Estimate 
fe_result = lm.estimate(y_dot, x_dot, transform='fe', T=T, robust_se=True)
lm.print_table((label_y, label_x_dot), fe_result, title="Fixed Effects", floatfmt='.4f')

Fixed Effects
Dependent variable: Log deflated sales

           Beta      Se    t-values
-------  ------  ------  ----------
Capital  0.1546  0.0299      5.1630
Labor    0.6942  0.0417     16.6674
R² = 0.477
σ² = 0.018


In [8]:
# CRS test for the fixed-effects output regression
W_fe, crit_fe, pval_fe = crs_test(fe_result)
print(f"CRS Wald test (FE): {W_fe:.4f}")
print(f"Critical value (5%): {crit_fe:.4f}")
print(f"p-value: {pval_fe:.4f}")
fe_output_result = fe_result


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 1 is different from 2)

In [ ]:
# Transform the data
D_T = - np.eye(T-1, T) + np.eye(T-1, T, k=1)
y_diff = lm.perm(D_T, y)
x_diff = lm.perm(D_T, x)

# Remove the columns that are only zeroes
x_diff, label_x_diff = remove_zero_columns(x_diff, label_x)

# Estimate 
fd_result = lm.estimate(y_diff, x_diff, transform='fd', T=T-1, robust_se=True)
lm.print_table((label_y, label_x_diff), fd_result, title="First Difference", floatfmt='.4f')

In [ ]:
# CRS test for the first-difference output regression
W_fd, crit_fd, pval_fd = crs_test_fd(fd_result)
print(f"CRS Wald test (FD): {W_fd:.4f}")
print(f"Critical value (5%): {crit_fd:.4f}")
print(f"p-value: {pval_fd:.4f}")
fd_output_result = fd_result


In [ ]:
# Transform the data
P_T = np.ones((1,T)) * 1/T
y_mean = lm.perm(P_T, y)
x_mean = lm.perm(P_T, x)

# Estimate 
be_result = lm.estimate(y_mean, x_mean, transform='be', T=T)
lm.print_table((label_y, label_x), be_result, title="Between Estimator", floatfmt='.4f')

In [ ]:
# Calculate lambda (note lambda is a reserved keyword in Python, so we use _lambda instead)
sigma2_u = fe_result['sigma2']
sigma2_w = be_result['sigma2']
sigma2_c = sigma2_w - 1/T * sigma2_u
_lambda = 1 - np.sqrt(sigma2_u / (sigma2_u + T*sigma2_c))

# Print lambda 
print(f'Lambda is approximately equal to {_lambda.item():.4f}.')

In [ ]:
# Transform the data
P_T_full = np.ones((T, T)) / T
C_T = np.eye(T) - _lambda * P_T_full
y_re = lm.perm(C_T, y)
x_re = lm.perm(C_T, x)

# Estimate 
re_result = lm.estimate(y_re, x_re, transform='re', T=T, robust_se=True)
lm.print_table((label_y, label_x), re_result, title="Random Effects", floatfmt='.4f')

In [ ]:
# CRS test for the random-effects output regression
W_re, crit_re, pval_re = crs_test(re_result)
print(f"CRS Wald test (RE): {W_re:.4f}")
print(f"Critical value (5%): {crit_re:.4f}")
print(f"p-value: {pval_re:.4f}")
re_output_result = re_result


In [ ]:

# Unpack HAUSMAN test
b_fe = fe_result['b_hat'][1:3,:]      # Select only Capital and Labor coefficients
b_re = re_result['b_hat'][1:3,:]      # Select only Capital and Labor coefficients
cov_fe = fe_result['cov'][1:3,1:3]    # Select only Capital and Labor covariance
cov_re = re_result['cov'][1:3,1:3]    # Select only Capital and Labor covariance

# Calculate the test statistic
b_diff = b_fe - b_re
cov_diff = cov_fe - cov_re
H = b_diff.T @ la.inv(cov_diff) @ b_diff

# Find critical value and p-value at 5% significance level of chi^2 with M degrees of freedom
M = len(b_diff)
crit_val = chi2.ppf(0.95, M)
p_val = 1 - chi2.cdf(H.item(), M)

# Print the results
print(f'The Hausman test statistic is {H.item():.2f}.')
print(f'The critical value at a 5% significance level is {crit_val:.2f}.')
print(f'The p-value is {p_val:.18f}.')


denne sektion skal nok ind i py filen

In [9]:
import numpy as np
from numpy import linalg as la
import pandas as pd
from io import StringIO
from tabulate import tabulate
from matplotlib import pyplot as plt

#Supress Future Warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Import this weeks LinearModels .py file
import w2_LinearModels_post as lm
%load_ext autoreload
%autoreload 2

In [11]:
# First, import the data into numpy. 
data = pd.read_csv('firms.csv')
id_array = np.array(data.iloc[:, 0])

# Count how many persons we have. This returns a tuple with the unique IDs,
# and the number of times each person is observed.
unique_id = np.unique(id_array, return_counts=True)
N = unique_id[0].size
T = int(unique_id[1].mean())
year = np.array(data.iloc[:, 1], dtype=int)

In [13]:
# Load the rest of the data into arrays.
# Load the rest of the data into arrays.
y = data['lcap'].to_numpy().reshape(-1, 1)

# x needs to have a constant vector in the first row. How would you add this? 
# Note that the order is set to match the order of variables in the model.
x = np.column_stack([
    np.ones(N * T),
    data['lemp'].to_numpy(),
    data['ldsa'].to_numpy()
])

# Lets also make some variable names
label_y = 'Log capital'
label_x = [
    'Constant',
    'Log employment',
    'Log DSA'
]


In [ ]:
# Estimate coefficients
b_hat = lm.est_ols(y,x)


In [ ]:
# Calculate the residuals
resid = y - x @ b_hat

# Calculate estimate of variance of residuals
SSR = resid.T @ resid
K = x.shape[1]
sigma = SSR / (N*T - K)

# Calculate the variance-covariance matrix
cov = sigma * la.inv(x.T @ x)

# Calculate the standard errors 
# Make sure to output the result in a vector
se = np.sqrt(np.diag(cov)).reshape(-1,1)

#Print results
for label, b_k, se_k in zip(label_x, b_hat, se):
    print(f'{label:16}: {b_k[0]:7.4f}    ({se_k[0]:6.4f})')

In [ ]:
# Estimate model using OLS
ols_result = lm.estimate(y,x, N=N, T=T)

In [ ]:
# Create transformation matrix
def demeaning_matrix(T):
    Q_T = np.eye(T) - np.tile(1/T, (T, T))
    return Q_T

In [ ]:
# Transform the data
y_demean = lm.perm(Q_T, y)
x_demean = lm.perm(Q_T, x)


In [ ]:
# Create function to check rank of demeaned matrix, and return its eigenvalues.
def check_rank(x):
    print(f'Rank of demeaned x: {la.matrix_rank(x)}')
    lambdas, V = la.eig(x.T@x)
    np.set_printoptions(suppress=True)  # This is just to print nicely.
    print(f'Eigenvalues of within-transformed x: {lambdas.round(decimals=0)}')
    print(V)
    # Use eigen vectors to identify which variables are dropped.

# Check rank of demeaned x
check_rank(x_demean)

In [ ]:
# Choose variables to include in fixed effects model
x_demean = x_demean[:, 1:5]
label_x_fe = label_x[1:5]

In [ ]:
# Estimate FE OLS using the demeaned variables.
fe_result = lm.estimate(y_demean, x_demean, transform='fe', N=N, T=T)


In [ ]:
# Create transformation matrix
def fd_matrix(T):
    D_T = np.eye(T) - np.eye(T, k=-1)
    D_T = D_T[1:]
    return D_T

# Print the matrix
D_T = fd_matrix(T)
print(f'First differening matrix for T={T} \n', D_T)

In [ ]:
# Transform the data.
y_diff = lm.perm(D_T, y)
x_diff = lm.perm(D_T, x)

# Print x_diff
print(x_diff)

In [ ]:
# Check rank condition.
check_rank(x_diff)

In [ ]:
# Choose variables to include in fixed effects model
x_diff = x_diff[:, 1:5]
label_x_fd = label_x[1:5]

In [ ]:
# Estimate FE OLS using the demeaned variables.
fd_result = lm.estimate(y_diff, x_diff, transform='fd', N=N, T=T-1)

# Print results
lm.print_table((label_y, label_x_fd), fd_result, title='FD regression', floatfmt='.4f')

In [ ]:
# Make function to calculate the serial correlation
def serial_corr(y, x, T):
    # Calculate the residuals
    b_hat = lm.est_ols(y, x)
    e = y - x@b_hat
    
    # Create a lag transformation matrix
    L_T = np.eye(T, k=-1)
    L_T = L_T[1:]

    # Lag residuals
    e_l = lm.perm(L_T, e)

    # Create a transformation matrix that removes the first observation of each individual
    I_T = np.eye(T, k=0)
    I_T = I_T[1:]
    
    # Remove first observation of each individual
    e = lm.perm(I_T, e)
    
    # Calculate the serial correlation
    return lm.estimate(e, e_l,N=N,T=T-1)

In [ ]:
# Estimate serial correlation
corr_result = serial_corr(y_diff, x_diff, T-1)

# Print results
label_ye = 'OLS residual, e\u1d62\u209c'
label_e = ['e\u1d62\u209c\u208B\u2081']
lm.print_table(
    (label_ye, label_e), corr_result, 
    title='Serial Correlation', floatfmt='.4f'
)

In [ ]:
# Lead employment
F_T = np.eye(T, k=1)
F_T = F_T[:-1]

employment_lead = lm.perm(F_T, x[:, 1].reshape(-1, 1))

In [ ]:
# Remove the last observed year for every individual
I_T = np.eye(T, k=0)
I_T = I_T[:-1]

x_exo = lm.perm(I_T, x)
y_exo = lm.perm(I_T, y)

In [ ]:
# Add employment_lead to x_exo
x_exo = np.hstack((x_exo, employment_lead))

# Within transform the data
Q_T = demeaning_matrix(T - 1)
yw_exo = lm.perm(Q_T, y_exo)
xw_exo = lm.perm(Q_T, x_exo)

# Select variables
xw_exo = np.hstack((xw_exo[:, 1:5], xw_exo[:, -1].reshape(-1, 1)))

In [ ]:
# Estimate model
exo_test = lm.estimate(yw_exo, xw_exo, N=N, T=T - 1, transform='fe')

# Print results
label_exo = label_x_fe + ['Employment lead']
lm.print_table((label_y, label_exo), exo_test, title='Exogeneity test', floatfmt='.4f')
print("Critical value (5%):", crit_val)

In [ ]:
# Create lead of employment
F_T = np.eye(T, k=1)[:-1]  # shifts forward, drops last row
employment_lead = lm.perm(F_T, x[:, 2].reshape(-1, 1))  # employment is column 2

# Drop last year for everyone (so dimensions match)
I_T = np.eye(T)[:-1]
x_exo = lm.perm(I_T, x)
y_exo = lm.perm(I_T, y)  # <-- now y is ldsa

# Add employment_lead to regressors
x_exo = np.hstack((x_exo, employment_lead))

# Within transform (demean)
Q_T = demeaning_matrix(T - 1)
yw_exo = lm.perm(Q_T, y_exo)
xw_exo = lm.perm(Q_T, x_exo)

# Keep the usual regressors + the lead
xw_exo = np.hstack((xw_exo[:, 1:3], xw_exo[:, -1].reshape(-1, 1)))  
# columns: capital, employment, lead employment

# Estimate FE model with the lead
exo_test = lm.estimate(yw_exo, xw_exo, N=N, T=T-1, transform='fe')

# Print results
label_exo = ["Capital", "Employment", "Lead employment"]
lm.print_table(("Output (ldsa)", label_exo), exo_test, title="Strict exogeneity test on FE", floatfmt=".4f")
print("Critical value (5%):", crit_val)

In [ ]:
# Create FD transformation matrix
D_T = np.eye(T-1, T, k=1) - np.eye(T-1, T)

# Transform y and x with FD
y_fd = lm.perm(D_T, y)
x_fd = lm.perm(D_T, x)

# Create lead of Δemployment (column 2 of x_fd is Δemployment)
F_T_fd = np.eye(T-2, T-1, k=1)   # shifts forward in differences
employment_lead_fd = lm.perm(F_T_fd, x_fd[:, 2].reshape(-1, 1))

# Drop last FD observation for alignment
I_T_fd = np.eye(T-1)[:-1]
x_fd_exo = lm.perm(I_T_fd, x_fd)
y_fd_exo = lm.perm(I_T_fd, y_fd)

# Add lead of Δemployment
x_fd_exo = np.hstack((x_fd_exo, employment_lead_fd))

# Keep same regressors: Δcapital, Δemployment, Δemployment_lead
x_fd_exo = np.hstack((x_fd_exo[:, 1:3], x_fd_exo[:, -1].reshape(-1, 1)))

# Estimate FD model
exo_test_fd = lm.estimate(y_fd_exo, x_fd_exo, N=N, T=T-2, transform="fd")

# Print results
labels_fd = ["ΔCapital", "ΔEmployment", "Lead ΔEmployment"]
lm.print_table(("ΔOutput (ldsa)", labels_fd), exo_test_fd,
               title="Strict Exogeneity Test (FD)", floatfmt=".4f")


In [14]:
from w3_LinearModels import fd_exogeneity_lead_test


# FD test in log differences (assignment style)
fd_exogeneity_lead_test(y, x, N, T, cap_col=1, emp_col=2, logs=True, drop_zeros=True)


FD Exogeneity Test (lead-variable)
Model: Δ log y_it = b1 Δ log K_it + b2 Δ log L_it + b3 Δ log L_{i,t+1} + Δu_it

              Variable        Beta          SE         t
                 const     -0.0000      0.0035     -0.01
          Δlog Capital      0.0753      0.0345      2.18
       Δlog Employment     -0.0015      0.0241     -0.06
  Lead Δlog Employment      0.0338      0.0192      1.76

Test H0: b3 (Lead term) = 0 → t=1.76, p=0.08036 (df clustered=141)
→ Do NOT reject exogeneity in FD.
